In [78]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date, ForeignKey, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship
from datetime import datetime, timedelta
import json
import re
import numpy as np
from functools import reduce
import requests
from api_keys import email, passw #Please add to .gitnore file your own individual usernames and passwords. Also set up your account on the website with a watched area of your desire.

In [79]:
browser = Browser('chrome')
url = "https://housesigma.com/listings/watched-area-and-community"
browser.visit(url)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [80]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)
email = email
passw = passw
login = (email) 
email_input.send_keys(login)

In [81]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [82]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()

In [83]:
browser.is_element_present_by_css('.pc-listing-card', wait_time=10)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [89]:
data_list = []  

articles = soup_obj.find_all('article', class_='pc-listing-card')

for article in articles:
    data = {}

    # Address
    address_element = article.find('h3', class_='address')
    data['Address'] = address_element.text.strip() if address_element else None

    # Neighbourhood
    text1_element = article.find('span', class_='text1')
    data['Neighbourhood'] = text1_element.text.strip() if text1_element else None

    json_scripts = article.find_all('script', class_='hs-script-home-struct', type='application/ld+json')


    for script in json_scripts:
        json_data = script.string
        if json_data:
            json_dict = json.loads(json_data)
            
            # Long and Lat and floor size
            if 'floorSize' in json_dict:
                data['Floor Size'] = json_dict['floorSize']['value']
            if 'geo' in json_dict:
                data['Latitude'] = json_dict['geo']['latitude']
                data['Longitude'] = json_dict['geo']['longitude']

    # Date 
    date_preview_element = article.find('div', class_='date-preview')
    data['Date of Status'] = date_preview_element.text.strip() if date_preview_element else None

    # Price
    highlight_element = article.find('span', class_='highlight')
    data['Price Listed'] = highlight_element.text.strip() if highlight_element else None
   
    # Status
    status_element = article.select_one('div[class^="status-type"]')
    data['Status'] = status_element.text.strip() if status_element else None

    # Sold Price (if status is "Sold" or "Sold Conditional")
    if data['Status'] in ["Sold", "Sold Conditional"]:
        sold_price_element_special = article.select_one('div.price-area span.special')
        sold_price_element_highlight_special = article.select_one('div.price-area span.highlight.special')

        if sold_price_element_special:
            data['Sold Price'] = sold_price_element_special.text.strip()
        elif sold_price_element_highlight_special:
            data['Sold Price'] = sold_price_element_highlight_special.text.strip()
        else:
            data['Sold Price'] = None
    else:
        data['Sold Price'] = 0
    
    # Type of house
    type_element = article.find('p', class_='type')
    data['Type of House'] = type_element.text.strip() if type_element else None

    p_elements = article.find_all('p')
    for p in p_elements:
        text = p.get_text()

        # Bathroom
        if re.search(r'\b(?:bathroom)\b', text):
            bathroom_info = re.findall(r'\d+', text)
            if bathroom_info:
                data['Bathrooms'] = int(bathroom_info[0])

        # Bedroom
        if re.search(r'\b(?:bedroom)\b', text):
            bedroom_info = re.findall(r'\d+', text)
            if bedroom_info:
                data['Bedrooms'] = int(bedroom_info[0])
        
        # Garage
        if re.search(r'\b(?:garage)\b', text):
            garage_info = re.findall(r'\d+', text)
            if garage_info:
                data['Garage'] = int(garage_info[0]) 
                
    data_list.append(data)


for i, data in enumerate(data_list, 1):
    print(f"{i}: {data}")


1: {'Address': '306 - 390 Upper Middle Rd E , Oakville - College Park', 'Neighbourhood': 'College Park', 'Floor Size': 1899, 'Latitude': -79.70008, 'Longitude': 43.47632, 'Date of Status': '1 hour ago', 'Price Listed': '$1,499,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo Apt', 'Bedrooms': 2, 'Bathrooms': 3, 'Garage': 3}
2: {'Address': '804 - 1905 Pilgrims Way , Oakville - Glen Abbey', 'Neighbourhood': 'Glen Abbey', 'Floor Size': 1499, 'Latitude': -79.72874, 'Longitude': 43.4439, 'Date of Status': '1 hour ago', 'Price Listed': '$819,000', 'Status': 'Sold Conditional', 'Sold Price': '$819,000', 'Type of House': 'Condo Apt', 'Bedrooms': 2, 'Bathrooms': 2, 'Garage': 1}
3: {'Address': '120 - 95 Dundas St W , Oakville - Rural Oakville', 'Neighbourhood': 'Rural Oakville', 'Floor Size': 1099, 'Latitude': -79.7324946, 'Longitude': 43.476356, 'Date of Status': '1 hour ago', 'Price Listed': '$749,900', 'Status': 'Terminated', 'Sold Price': 0, 'Type of House': 'Condo Apt', 

In [90]:
def convert_relative_timestamp(relative_timestamp):
    if " hours ago" in relative_timestamp:
        hours = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(hours=hours)
        return absolute_timestamp.date()
    elif " days ago" in relative_timestamp:
        days = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(days=days)
        return absolute_timestamp.date()
    else:
        return datetime.now().date()


df = pd.DataFrame(data_list)
df['City'] = df['Address'].str.split(',').str[1].str.split(' - ').str[0]
df['Address'] = df['Address'].str.split(',').str[0]
df['Date of Status'] = df['Date of Status'].apply(convert_relative_timestamp)

df['Sold Price'] = df['Sold Price'].str.replace('$', '').str.replace(',', '').astype(float)
df['Price Listed'] = df['Price Listed'].str.replace('$', '').str.replace(',', '').astype(float)
df

,Address,Neighbourhood,Floor Size,Latitude,Longitude,Date of Status,Price Listed,Status,Sold Price,Type of House,Bedrooms,Bathrooms,Garage,City
0,306 - 390 Upper Middle Rd E,College Park,1899.0,-79.700080,43.476320,2024-02-05,1499000.0,For Sale,NaN,Condo Apt,2,3,3,Oakville
1,804 - 1905 Pilgrims Way,Glen Abbey,1499.0,-79.728740,43.443900,2024-02-05,819000.0,Sold Conditional,819000.0,Condo Apt,2,2,1,Oakville
2,120 - 95 Dundas St W,Rural Oakville,1099.0,-79.732495,43.476356,2024-02-05,749900.0,Terminated,NaN,Condo Apt,2,2,2,Oakville
3,1177 Summerlea St,Bronte East,NaN,-79.698870,43.422370,2024-02-05,3688800.0,For Sale,NaN,Detached,4,5,2,Oakville
4,2291 Colbeck St,West Oak Trails,1300.0,-79.764550,43.429860,2024-02-05,1090000.0,For Sale,NaN,Freehold Townhouse,3,2,1,Oakville
5,1307 - 297 Oak Walk Drive,Uptown Core,655.0,-79.718298,43.486047,2024-02-05,625000.0,For Sale,NaN,Condo/Apt Unit,1,1,1,Oakville
6,2048 Ardleigh Road,Eastlake,4500.0,-79.644237,43.472015,2024-02-05,7499000.0,For Sale,NaN,Single Family Residence,4,7,2,Oakville
7,107 - 269 Georgian Dr,Uptown Core,1099.0,-79.715680,43.481330,2024-02-05,749900.0,For Sale,NaN,Condo Townhouse,2,2,1,Oakville
8,7 - 2300 Brays Lane,Glen Abbey,1299.0,-79.738670,43.426420,2024-02-05,809888.0,Sold Conditional,809888.0,Condo Townhouse,3,2,1,Oakville
9,339 - 2485 Taunton Rd,Uptown Core,849.0,-79.719380,43.486300,2024-02-05,699000.0,For Sale,NaN,Condo Apt,2,2,1,Oakville


In [91]:
weather_data_oakville = 'Resources/Oakville_Historical_Weather.csv'
selected_columns = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE','LOCAL_DATE', 'STATION_NAME', 'COOLING_DEGREE_DAYS', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_YEAR', 'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE', 'ID']
df_weather = pd.read_csv(weather_data_oakville, usecols=selected_columns, encoding="ISO-8859-1", on_bad_lines='skip', low_memory=True, engine='python')
df_weather = df_weather.dropna(subset=['MIN_TEMPERATURE'])
df_weather = df_weather.reset_index(drop=True)
columns_to_fill = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE', 'COOLING_DEGREE_DAYS', 
                   'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 
                   'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE']
df_weather[columns_to_fill] = df_weather[columns_to_fill].fillna(0)
df_weather['LOCAL_DATE'] = pd.to_datetime(df_weather['LOCAL_DATE']).dt.date
df_weather.sample(10)

,HEATING_DEGREE_DAYS,MIN_TEMPERATURE,LOCAL_DATE,STATION_NAME,COOLING_DEGREE_DAYS,LOCAL_MONTH,LOCAL_DAY,LOCAL_YEAR,TOTAL_PRECIPITATION,SNOW_ON_GROUND,MEAN_TEMPERATURE,TOTAL_SNOW,TOTAL_RAIN,MAX_TEMPERATURE,ID
1209,0.0,19.5,1993-09-14,OAKVILLE GERARD,6.8,9,14,1993,0.6,0.0,24.8,0.0,0.6,30.0,6155PD4.1993.9.14
1834,5.2,10.5,1995-06-08,OAKVILLE GERARD,0.0,6,8,1995,0.0,0.0,12.8,0.0,0.0,15.0,6155PD4.1995.6.8
3078,10.7,6.0,1998-11-11,OAKVILLE GERARD,0.0,11,11,1998,0.0,0.0,7.3,0.0,0.0,8.5,6155PD4.1998.11.11
2706,8.2,2.5,1997-10-31,OAKVILLE GERARD,0.0,10,31,1997,2.0,0.0,9.8,0.0,2.0,17.0,6155PD4.1997.10.31
2268,0.0,15.5,1996-08-17,OAKVILLE GERARD,2.8,8,17,1996,0.0,0.0,20.8,0.0,0.0,26.0,6155PD4.1996.8.17
2550,5.5,4.0,1997-05-28,OAKVILLE GERARD,0.0,5,28,1997,0.0,0.0,12.5,0.0,0.0,21.0,6155PD4.1997.5.28
326,21.0,-8.5,1991-03-12,OAKVILLE GERARD,0.0,3,12,1991,0.0,0.0,-3.0,0.0,0.0,2.5,6155PD4.1991.3.12
824,3.7,8.0,1992-07-22,OAKVILLE GERARD,0.0,7,22,1992,12.2,0.0,14.3,0.0,12.2,20.5,6155PD4.1992.7.22
1364,21.5,-8.0,1994-02-16,OAKVILLE GERARD,0.0,2,16,1994,0.0,34.0,-3.5,0.0,0.0,1.0,6155PD4.1994.2.16
135,0.0,16.5,1990-09-02,OAKVILLE GERARD,5.5,9,2,1990,0.0,0.0,23.5,0.0,0.0,30.5,6155PD4.1990.9.2


In [92]:
Base = declarative_base()
#Base.metadata.clear()

C:\Users\qwert\AppData\Local\Temp\ipykernel_3232\2374365428.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [93]:
class House(Base):
    __tablename__ = 'houses'
    id = Column(Integer, primary_key=True)
    address = Column(String)
    status = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    floor_size = Column(Integer)
    bedrooms = Column(Integer)  
    bathrooms = Column(Integer) 
    garage = Column(Integer)
    city = Column(String)
    type_of_house = Column(String)  
    date_listed = Column(Date)
    neighbourhood = Column(String)
    price = Column(Float)

class Sold(Base):
    __tablename__ = 'sold'
    id = Column(Integer, primary_key=True)
    address = Column(String, ForeignKey('houses.address'))
    price = Column(Float)
    status = Column(String)
    date_listed = Column(Date)
    sold_price = Column(Float)
    type_of_house = Column(String)  
    city = Column(String)
    neighbourhood = Column(String)

class WeatherData(Base):
    __tablename__ = 'weather_data'

    id = Column(String, primary_key=True, unique=True) 
    heating_degree_days = Column(Float)
    min_temperature = Column(Float)
    local_date = Column(String)
    station_name = Column(String)
    cooling_degree_days = Column(Float)
    local_month = Column(Integer)
    local_day = Column(Integer)
    local_year = Column(Integer)
    total_precipitation = Column(Float)
    snow_on_ground = Column(Float)
    mean_temperature = Column(Float)
    total_snow = Column(Float)
    total_rain = Column(Float)
    max_temperature = Column(Float)

In [94]:
engine = create_engine('sqlite:///housing_sales_db.db')
session = Session(bind=engine)
#Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
session.commit()
session.close()

In [95]:
engine = create_engine('sqlite:///housing_sales_db.db')
session = Session(bind=engine)

In [96]:
for data in data_list:
    address = data.get('Address')
    if address:
        house_exists = session.query(House).filter_by(address=address).first()

    if house_exists:
        house_exists.status = data.get('Status')
        house_exists.date_listed = convert_relative_timestamp(data.get('Date of Status'))
    else:
        address_parts = address.split(',')
        city = address_parts[1].split(' - ')[0].strip()
        address = address_parts[0]

        new_house = House(
            address=address,
            status=data.get('Status'),
            latitude=data.get('Latitude'),
            longitude=data.get('Longitude'),
            floor_size=data.get('Floor Size'),
            bedrooms=data.get('Bedrooms'),
            bathrooms=data.get('Bathrooms'),
            garage=data.get('Garage'),
            city=city,  
            type_of_house=data.get('Type of House'),
            date_listed=convert_relative_timestamp(data.get('Date of Status')),
            neighbourhood=data.get('Neighbourhood'),
            price=float(data.get('Price Listed').replace('$', '').replace(',', ''))
        )
        session.add(new_house)

In [97]:
for data in data_list:
    address = data.get('Address')
    status = data.get('Status')

    if status in ["Sold", "Sold Conditional"]:
        sold_house = session.query(Sold).filter_by(address=address).first()

        if sold_house:
            sold_house.status = status
            sold_house.date_listed = convert_relative_timestamp(data.get('Date of Status'))
            sold_house.sold_price = float(str(data.get('Sold Price')).replace('$', '').replace(',', ''))
        else:
            address_parts = address.split(',')
            city = address_parts[1].split(' - ')[0].strip()
            address = address_parts[0]
            new_sold_house = Sold(
                address=address,
                price=float(data.get('Price Listed').replace('$', '').replace(',', '')),
                status=status,
                date_listed=convert_relative_timestamp(data.get('Date of Status')),
                sold_price=float(str(data.get('Sold Price')).replace('$', '').replace(',', '')),
                city=city,
                type_of_house=data.get('Type of House'),
                neighbourhood=data.get('Neighbourhood')
            )
            session.add(new_sold_house)

In [98]:
weather_data_instances = []

for index, row in df_weather.iterrows():
    weather_data_instance = WeatherData(
        id=row['ID'],
        heating_degree_days=row['HEATING_DEGREE_DAYS'],
        min_temperature=row['MIN_TEMPERATURE'],
        local_date=row['LOCAL_DATE'],
        station_name=row['STATION_NAME'],
        cooling_degree_days=row['COOLING_DEGREE_DAYS'],
        local_month=row['LOCAL_MONTH'],
        local_day=row['LOCAL_DAY'],
        local_year=row['LOCAL_YEAR'],
        total_precipitation=row['TOTAL_PRECIPITATION'],
        snow_on_ground=row['SNOW_ON_GROUND'],
        mean_temperature=row['MEAN_TEMPERATURE'],
        total_snow=row['TOTAL_SNOW'],
        total_rain=row['TOTAL_RAIN'],
        max_temperature=row['MAX_TEMPERATURE']
    )
    weather_data_instances.append(weather_data_instance)

session.add_all(weather_data_instances)

In [99]:
session.commit()

In [100]:
sold_count = session.query(func.count(Sold.id)).scalar()
house_count = session.query(func.count(House.id)).scalar()
weather_count = session.query(func.count(WeatherData.id)).scalar()
print("Number of records in Weather table:", weather_count)
print("Number of records in Sold table:", sold_count)
print("Number of records in House table:", house_count)

Number of records in Weather table: 5894
Number of records in Sold table: 2
Number of records in House table: 10


In [101]:
session.close()

In [102]:
browser.quit()